In [1]:
# Fake News Prediction

In [2]:
## Preparation


## EDA

### Number of character

### Number of word

### Number of sentence

### Stopwords Analysis

### Punctuation Analysis

### Bi-gram Analysis



## Preprocessing

### Cleaning

### Splitting Train & Test


## Feature Extraction

### Bag of Words


## GloVe Vectorization


## Modelling

### Random Forest

### Naive Bayes

### Neural Network